In [ ]:
import pandas as pd
import re
from datetime import datetime, timedelta
import os

# SAMI 파일에서 시작 시간과 FPS 정보 추출
def extract_info_from_sami(sami_file_path):
    with open(sami_file_path, 'r', encoding='utf-8') as file:
        content = file.read()
    
    # 시작 시간 추출
    start_time_pattern = re.compile(r'<SYNC Start=0000><P Class=SUBTTL>(\d{4}/\d{2}/\d{2} \d{2}:\d{2}:\d{2})')
    start_time_match = start_time_pattern.search(content)
    if start_time_match:
        start_time = datetime.strptime(start_time_match.group(1), "%Y/%m/%d %H:%M:%S")
    else:
        raise ValueError(f"시작 시간을 찾을 수 없습니다: {sami_file_path}")
    
    # FPS 정보 추출
    fps_pattern = re.compile(r'<SYNC Start=(\d+)><P Class=SUBTTL>.*?(\d+)FPS')
    matches = fps_pattern.findall(content)
    
    fps_info = []
    for match in matches:
        time_ms = int(match[0])
        fps = int(match[1])
        time_s = time_ms / 1000
        fps_info.append({"time_s": time_s, "fps": fps})
    
    return start_time, fps_info

# 프레임 번호를 기준으로 시간을 찾는 함수
def find_time_for_frame(frame_number, start_time, fps_info):
    elapsed_time = 0
    total_frames = 0
    
    for info in fps_info:
        total_frames += info['fps']
        if frame_number <= total_frames:
            fractional_time = (frame_number - (total_frames - info['fps'])) / info['fps']
            actual_time = start_time + timedelta(seconds=elapsed_time + fractional_time)
            return actual_time.strftime("%H:%M:%S.%f")[:-3]
        elapsed_time += 1

# 파일 이름 패턴을 기반으로 SAMI 파일과 CSV 파일을 처리
def process_files(sami_file_path, csv_file_path):
    start_time, fps_info = extract_info_from_sami(sami_file_path)

    # CSV 파일 읽기
    df = pd.read_csv(csv_file_path)

    # TIME 열 추가
    df['TIME'] = df['frame'].apply(lambda x: find_time_for_frame(x, start_time, fps_info))

    # 결과 CSV 파일 저장
    output_folder = 'C:/Users/PC/Desktop/caffein/montana/result_0901_0912_time'
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    base_name = os.path.splitext(os.path.basename(csv_file_path))[0]
    output_file_path = os.path.join(output_folder, f'{base_name}_time.csv')
    df.to_csv(output_file_path, index=False)

# 폴더 내에서 SAMI 파일과 CSV 파일을 매칭하는 함수
def find_matching_files(sami_folder, csv_folder):
    sami_files = [f for f in os.listdir(sami_folder) if f.endswith('.smi')]
    csv_files = [f for f in os.listdir(csv_folder) if f.endswith('_res.csv')]

    matched_files = []
    
    for sami_file in sami_files:
        base_name = os.path.splitext(sami_file)[0]  # 예: '0901_1'
        csv_file = f"{base_name}_res.csv"
        
        if csv_file in csv_files:
            matched_files.append((os.path.join(sami_folder, sami_file), os.path.join(csv_folder, csv_file)))
    
    return matched_files

# SAMI 파일과 CSV 파일이 있는 폴더 경로
sami_folder = 'C:/Users/PC/Desktop/caffein/montana/video_0901_0912/0901_0912_smi'
csv_folder = 'C:/Users/PC/Desktop/caffein/montana/result_0901_0912'

# SAMI 파일과 CSV 파일을 매칭하고 처리
matched_files = find_matching_files(sami_folder, csv_folder)

for sami_file, csv_file in matched_files:
    process_files(sami_file, csv_file)